# Importing and loading necessary libraries

In [5]:
# !pip install opencv-python
# ! pip install imutils

In [6]:
import numpy as np
import cv2
import random
import os

from imutils import paths


from keras.models import Sequential
from keras.layers.core import Activation
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras import applications
from keras.applications.vgg16 import VGG16
# from keras.applications.vgg19 import VGG19
# from keras.applications.resnet50 import ResNet50
# from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import load_model
from keras import backend as K
from collections import defaultdict

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, precision_recall_fscore_support, average_precision_score, confusion_matrix
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/Users/noor-4785/Documents/training_and_developement/vit_fdp/temp_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/noor-4785/Documents/training_and_developement/vit_fdp/temp_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/noor-4785/Documents/training_and_developement/vit_fdp/temp_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be underst

# Create dataset

In [ ]:
# data/foldername

# Load and preprocess data

In [48]:
#dataset path

# path = 'data/cat_and_dog_100'
path = 'data/cat_and_dog_1000'

In [49]:
# Initialise required variables
data = []
labels = []
features = []
RANDOM_STATE = 42

In [51]:
# Initialise required hyperparameters
IMAGE_DIM = 48
TEST_SPLIT = 0.20
BATCH_SIZE = 32
EPOCHS = 5
INIT_LR = 1e-3

In [52]:
# Extracting the class information from the given folder : Class name and corresponding number of files
class_dir_paths = os.listdir(path)
class_dict = {}
idx = 0

for class_dir_path in class_dir_paths:
    if (os.path.isdir(os.path.join(path, class_dir_path))):
        class_name = class_dir_path
        class_dict[class_name] = idx
        idx += 1

print("Class information : ", class_dict)

Class information :  {'cat': 0, 'dog': 1}


In [53]:
# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images(path)))
random.seed(RANDOM_STATE)
random.shuffle(imagePaths)

In [54]:
len(imagePaths)

2001

In [55]:
imagePaths[0]

'data/cat_and_dog_1000/cat/cat.813.jpg'

In [56]:
# loop over the input images
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIM, IMAGE_DIM))
    image = img_to_array(image)
    feature = image_to_feature_vector(image)
    data.append(image)
    features.append(feature)

    # extract the class label from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(class_dict[label])

print("Total number of images readed, ", len(data))
print("Total number of labels extracted, ", len(labels))

Total number of images readed,  2001
Total number of labels extracted,  2001


# Simple Neural network

In [57]:
len(features[0])

3072

In [58]:
features = np.array(features, dtype="float") / 255.0

(ttrainX, ttestX, ttrainY, ttestY) = train_test_split(
    features, labels, test_size=TEST_SPLIT, random_state=RANDOM_STATE)

# convert the labels from integers to vectors
ttrainY = to_categorical(ttrainY, num_classes=len(class_dict))
ttestY = to_categorical(ttestY, num_classes=len(class_dict))

print("Number of image in training set ", len(ttrainX), len(ttrainY))
print("Number of image in tesing set ", len(ttestX), len(ttestY))

Number of image in training set  1600 1600
Number of image in tesing set  401 401


In [59]:
# define the architecture of the network
model = Sequential()
model.add(Dense(768, input_dim=3072, activation="relu"))
model.add(Dense(384, activation="relu"))
model.add(Dense(2))
model.add(Activation("softmax"))

In [60]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 768)               2360064   
_________________________________________________________________
dense_10 (Dense)             (None, 384)               295296    
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 770       
_________________________________________________________________
activation_6 (Activation)    (None, 2)                 0         
Total params: 2,656,130
Trainable params: 2,656,130
Non-trainable params: 0
_________________________________________________________________


In [61]:
# Compiling the model
sgd = SGD(lr=0.01)
model.compile(loss="binary_crossentropy", optimizer=sgd, metrics=["accuracy"])

In [62]:
print("[INFO] training network...")
nn_history = model.fit(
    ttrainX,
    ttrainY,
    epochs=EPOCHS,
    batch_size=32,
    verbose=1,
    validation_data=(ttestX, ttestY))

[INFO] training network...
Train on 1600 samples, validate on 401 samples
Epoch 1/5
1600/1600 [==============================] - 1s 805us/step - loss: 0.7250 - acc: 0.5206 - val_loss: 0.6955 - val_acc: 0.5037
Epoch 2/5
1600/1600 [==============================] - 1s 545us/step - loss: 0.6889 - acc: 0.5613 - val_loss: 0.6543 - val_acc: 0.6185
Epoch 3/5
1600/1600 [==============================] - 1s 567us/step - loss: 0.6649 - acc: 0.6038 - val_loss: 0.6766 - val_acc: 0.5511
Epoch 4/5
1600/1600 [==============================] - 1s 582us/step - loss: 0.6540 - acc: 0.6125 - val_loss: 0.6405 - val_acc: 0.6758
Epoch 5/5
1600/1600 [==============================] - 1s 601us/step - loss: 0.6380 - acc: 0.6500 - val_loss: 0.6360 - val_acc: 0.6608


In [63]:
print_score(model, ttrainX, ttrainY, ttestX, ttestY)

401/401 [==============================] - 0s 104us/step
************ Training Metrics ************
Accuracy =  0.704375
Confusion Matrix
[[543 243]
 [230 584]]
************ Testing Metrics ************
Accuracy =  0.660847880893812
Confusion Matrix
[[140  75]
 [ 61 125]]


In [64]:
# plot_save_metrics(nn_history, 'nn_plot')

# Convolutional Neural network

In [65]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [66]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=TEST_SPLIT, random_state=RANDOM_STATE)

print("Number of image in training set ", len(trainX), len(trainY))
print("Number of image in tesing set ", len(testX), len(testY))

Number of image in training set  1600 1600
Number of image in tesing set  401 401


In [67]:
# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=len(class_dict))
testY = to_categorical(testY, num_classes=len(class_dict))

In [68]:
def simple_net(width, height, depth, classes):

    finalAct = "softmax" if classes > 2 else "sigmoid"

    # initialize the model
    model = Sequential()
    inputShape = (height, width, depth)

    # if we are using "channels first", update the input shape
    if K.image_data_format() == "channels_first":
        inputShape = (depth, height, width)

    # first set of CONV => RELU => POOL layers
    model.add(Conv2D(20, (5, 5), padding="same", input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # second set of CONV => RELU => POOL layers
    model.add(Conv2D(50, (5, 5), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # first (and only) set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation("relu"))

    # softmax classifier
    model.add(Dense(classes))
    model.add(Activation(finalAct))

    # return the constructed network architecture
    return model

In [69]:
aug = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest")

In [70]:
model = simple_net(IMAGE_DIM, IMAGE_DIM, 3, len(class_dict))

In [71]:
# Choose the optimizer
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

# Complile the model
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [72]:
print("[INFO] training network...")
H = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1)

[INFO] training network...
Epoch 1/25
50/50 [==============================] - 6s 115ms/step - loss: 0.7178 - acc: 0.5081 - val_loss: 0.6885 - val_acc: 0.5661
Epoch 2/25
50/50 [==============================] - 5s 95ms/step - loss: 0.6920 - acc: 0.5247 - val_loss: 0.6881 - val_acc: 0.5599
Epoch 3/25
50/50 [==============================] - 5s 91ms/step - loss: 0.6837 - acc: 0.5837 - val_loss: 0.6694 - val_acc: 0.5636
Epoch 4/25
50/50 [==============================] - 5s 90ms/step - loss: 0.6755 - acc: 0.5722 - val_loss: 0.6688 - val_acc: 0.5586
Epoch 5/25
50/50 [==============================] - 5s 90ms/step - loss: 0.6643 - acc: 0.6038 - val_loss: 0.6301 - val_acc: 0.6858
Epoch 6/25
50/50 [==============================] - 5s 95ms/step - loss: 0.6348 - acc: 0.6594 - val_loss: 0.5796 - val_acc: 0.7419
Epoch 7/25
50/50 [==============================] - 5s 92ms/step - loss: 0.6270 - acc: 0.6466 - val_loss: 0.5813 - val_acc: 0.7120
Epoch 8/25
50/50 [==============================] - 5s 

In [73]:
print_score(model,trainX, trainY, testX, testY)

401/401 [==============================] - 0s 443us/step
************ Training Metrics ************
Accuracy =  0.795
Confusion Matrix
[[645 141]
 [190 624]]
************ Testing Metrics ************
Accuracy =  0.7855361590064375
Confusion Matrix
[[176  39]
 [ 47 139]]


## To save the model

In [24]:
model.save('keras_model')

## Load the saved model and do prediction

In [29]:
# load the image
image = cv2.imread('data/cat_and_dog_1000/cat/cat.813.jpg')
orig = image.copy()

# pre-process the image for classification
image = cv2.resize(image, (IMAGE_DIM, IMAGE_DIM))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

# load the trained convolutional neural network
print("[INFO] loading network...")
model = load_model('keras_model')


[INFO] loading network...


In [33]:
(cat, dog) = model.predict(image)[0]

In [36]:
# One with the highest value is our prediction
(cat, dog)

(0.4179721, 0.5898914)

# Transfer Learning

In [74]:
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(IMAGE_DIM, IMAGE_DIM, 3))

In [75]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [76]:
#Freeze all the layers
for layer in base_model.layers:
    layer.trainable = False

In [77]:
# Building model based on Function API
X = base_model.output
X = Flatten()(X)  

#Adding last FC layers based on hyperparams given

X = Dense(128, activation='relu')(X)
X = Dense(128, activation='relu')(X)

# for layer_param in hyperparameters['top_layers']:
# 	X = self.layers[layer_param[0]](layer_param[1], activation=layer_param[2])(X)

#Adding dropout
X = Dropout(0.5)(X)

#Adding the last layer for prediction
predictions = Dense(2, activation='sigmoid')(X)

#creating the final model
model = Model(base_model.input, predictions)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [78]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [79]:
print("[INFO] training network...")
H = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1)

[INFO] training network...
Epoch 1/5
50/50 [==============================] - 12s 231ms/step - loss: 0.6620 - acc: 0.6047 - val_loss: 0.6101 - val_acc: 0.6683
Epoch 2/5
50/50 [==============================] - 11s 228ms/step - loss: 0.5908 - acc: 0.6878 - val_loss: 0.5626 - val_acc: 0.7170
Epoch 3/5
50/50 [==============================] - 12s 231ms/step - loss: 0.5668 - acc: 0.7109 - val_loss: 0.5800 - val_acc: 0.6958
Epoch 4/5
50/50 [==============================] - 12s 237ms/step - loss: 0.5455 - acc: 0.7200 - val_loss: 0.5691 - val_acc: 0.7120
Epoch 5/5
50/50 [==============================] - 12s 241ms/step - loss: 0.5388 - acc: 0.7297 - val_loss: 0.5583 - val_acc: 0.7057


In [80]:
print_score(model,trainX, trainY, testX, testY)

401/401 [==============================] - 2s 6ms/step
************ Training Metrics ************
Accuracy =  0.7909375
Confusion Matrix
[[622 164]
 [170 644]]
************ Testing Metrics ************
Accuracy =  0.7057356602533202
Confusion Matrix
[[146  69]
 [ 48 138]]


# Finetuning

In [81]:
# Unfreeze only the last few layers
N_LAYERS_TO_FREEZE = 15

for layer in model.layers[:15]:
    layer.trainable = False

for layer in model.layers[15:]:
    layer.trainable = True
    
# Unfreeze all the layers
# for layer in model.layers:
#     layer.trainable = True

In [82]:
print("[INFO] Finetuning the network...")
model.compile(
    optimizer=SGD(lr=0.001, momentum=0.9),
    loss='binary_crossentropy',
    metrics=['accuracy'])

[INFO] Finetuning the network...


In [83]:
H_tune = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1)

Epoch 1/5
50/50 [==============================] - 21s 425ms/step - loss: 0.5571 - acc: 0.7116 - val_loss: 0.6375 - val_acc: 0.6421
Epoch 2/5
50/50 [==============================] - 20s 407ms/step - loss: 0.4950 - acc: 0.7531 - val_loss: 0.4837 - val_acc: 0.7581
Epoch 3/5
50/50 [==============================] - 21s 412ms/step - loss: 0.5020 - acc: 0.7588 - val_loss: 0.4659 - val_acc: 0.7544
Epoch 4/5
50/50 [==============================] - 21s 411ms/step - loss: 0.4694 - acc: 0.7484 - val_loss: 0.5017 - val_acc: 0.7494
Epoch 5/5
50/50 [==============================] - 21s 417ms/step - loss: 0.4336 - acc: 0.8003 - val_loss: 0.4653 - val_acc: 0.7706


In [84]:
print_score(model,trainX, trainY, testX, testY)

401/401 [==============================] - 2s 6ms/step
************ Training Metrics ************
Accuracy =  0.8684375
Confusion Matrix
[[676 110]
 [ 97 717]]
************ Testing Metrics ************
Accuracy =  0.770573566233428
Confusion Matrix
[[155  60]
 [ 31 155]]


# utils

In [19]:
def plot_save_metrics(model_history):

    # plot the training loss and accuracy
    plt.style.use("ggplot")
    plt.figure()
    N = EPOCHS
    plt.plot(
        np.arange(0, N), model_history.history["loss"], label="train_loss")
    plt.plot(
        np.arange(0, N), model_history.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), model_history.history["acc"], label="train_acc")
    plt.plot(
        np.arange(0, N), model_history.history["val_acc"], label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    return plt

In [20]:
def print_score(model, trainX, trainY, testX, testY):

    train_score = model.evaluate(trainX, trainY)
    train_predictions = model.predict(trainX)
    train_predictions = np.round(train_predictions)

    test_score = model.evaluate(testX, testY)
    test_predictions = model.predict(testX)
    test_predictions = np.round(test_predictions)

    print("************ Training Metrics ************")

#     print("Loss = ", train_score[0])
    print("Accuracy = ", train_score[1])
#     print("classification Report")
#     print(classification_report(trainY, train_predictions))
    print("Confusion Matrix")
    train_cm = confusion_matrix(
        trainY.argmax(axis=1), train_predictions.argmax(axis=1))
    print(train_cm)

    print("************ Testing Metrics ************")

#     print("Loss = ", test_score[0])
    print("Accuracy = ", test_score[1])
#     print("classification Report")
#     print(classification_report(testY, test_predictions))
    print("Confusion Matrix")
    test_cm = confusion_matrix(
        testY.argmax(axis=1), test_predictions.argmax(axis=1))
    print(test_cm)

In [21]:
# resize the image to a fixed size, then flatten the image into
# a list of raw pixel intensities

def image_to_feature_vector(image, size=(32, 32)):
    return cv2.resize(image, size).flatten()